In [1]:
client_id = ''
client_secret = ''

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 xynm62v1qqrn41ajhhwpcjp370ko5w


In [4]:
oauth_token = ''

In [5]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/rivers_gg"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '734906922', 'login': 'rivers_gg', 'display_name': 'rivers_gg', 'type': '', 'broadcaster_type': 'partner', 'description': 'gamer de closet (ya no tanto) cod warzone y otros juegos que no requieren talento // streams lunes-viernes 5pm hora centro de mexico', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/0e6f8782-d5b9-4a51-ae8a-9c952c213487-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2021-10-16T21:51:30Z'}]}


In [7]:
data = {'data': [{'id': '734906922', 'login': 'rivers_gg', 'display_name': 'rivers_gg', 'type': '', 'broadcaster_type': 'partner', 'description': 'gamer de closet (ya no tanto) cod warzone y otros juegos que no requieren talento // streams lunes-viernes 5pm hora centro de mexico', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/0e6f8782-d5b9-4a51-ae8a-9c952c213487-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2021-10-16T21:51:30Z'}]}
data_dict = data['data'][0]
id_canal =  list(data_dict.values())[0]
id_canal

'734906922'

In [8]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             50 non-null     int64         
 1   URL del VOD            50 non-null     object        
 2   Nombre del VOD         50 non-null     object        
 3   Duración               50 non-null     object        
 4   Fecha de inicio        50 non-null     datetime64[ns]
 5   Fecha de finalización  50 non-null     datetime64[ns]
 6   Visitas                50 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 14.3 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1891254465,https://www.twitch.tv/videos/1891254465,central_gamer_soriana_-_día_1,2h33m33s,2023-08-06 00:29:42,2023-08-06 00:29:42,802114
1,1889568223,https://www.twitch.tv/videos/1889568223,ola_apareci_en_cdmx_jsjajajsa,1h1m55s,2023-08-04 01:41:42,2023-08-04 01:41:42,299499
2,1889559424,https://www.twitch.tv/videos/1889559424,ola_apareci_en_cdmx_jsjajajsa,8m27s,2023-08-04 01:30:27,2023-08-04 01:30:27,95354
3,1888638567,https://www.twitch.tv/videos/1888638567,comprando_productos_de_shark_tank,3h38m7s,2023-08-03 00:29:54,2023-08-03 00:29:54,747075
4,1887636603,https://www.twitch.tv/videos/1887636603,probando_dulces_de_japon,3h14m41s,2023-08-01 21:31:53,2023-08-01 21:31:53,1043149


In [9]:
df.to_csv('twitch_api_rivers_gg_csv.csv', index=False)